## Importing danish texts

In [1]:
! pip install pandas openpyxl
! pip install pyarrow
!pip install nltk

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Read the Parquet file into a DataFrame
df = pd.read_parquet('test-00000-of-00001.parquet')

# Display the DataFrame
print(df)

                                                   text    label
0                     35-årig mand erkender drab [LINK]  neutral
1     BILLEDSERIE: Hjerteforeningens lokalkomite man...  neutral
2       Det må jo være fraregnet #LordBendtner 😉 [LINK]  neutral
3     Det er så godt, @USER! Psykiatriske disgnoser ...  positiv
4     Nøddesbos hånd i Herning-rundkørsel: Jeg er en...  positiv
...                                                 ...      ...
1042  @USER Jeg spørger bare, hvorfor dét er i orden...  negativ
1043                                    Så ENIG! [LINK]  positiv
1044  @USER Men det var jo ret sjovt :) som ikke er ...  negativ
1045  DAT fejrer 30 år med fly Det er i dag 30 år si...  positiv
1046  Eriksen har en glimrende statistik mod City [L...  positiv

[1047 rows x 2 columns]


In [4]:
df = df.drop(columns=df.columns[1])

In [5]:
import string

# Function to clean text
def clean_text(text):
    # Remove newlines and extra spaces
    text = text.replace('\n', ' ').strip()
    
    # Remove punctuation and digits
    # text = text.translate(str.maketrans('', '', string.punctuation + '0123456789'))
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# Apply the cleaning function to the DataFrame
df['Cleaned Text'] = df['text'].apply(clean_text)

# Display the cleaned DataFrame
print(df[['text', 'Cleaned Text']].head())

                                                text  \
0                  35-årig mand erkender drab [LINK]   
1  BILLEDSERIE: Hjerteforeningens lokalkomite man...   
2    Det må jo være fraregnet #LordBendtner 😉 [LINK]   
3  Det er så godt, @USER! Psykiatriske disgnoser ...   
4  Nøddesbos hånd i Herning-rundkørsel: Jeg er en...   

                                        Cleaned Text  
0                  35-årig mand erkender drab [link]  
1  billedserie: hjerteforeningens lokalkomite man...  
2    det må jo være fraregnet #lordbendtner 😉 [link]  
3  det er så godt, @user! psykiatriske disgnoser ...  
4  nøddesbos hånd i herning-rundkørsel: jeg er en...  


In [6]:
import re

# Function to tokenize text using regular expressions, keeping punctuation
def tokenize_with_re(text):
    # Use regex to capture words and punctuation marks
    tokens = re.findall(r'\b\w+\b|[.,!?;()]', text.lower())  # Capture words and common punctuation
    return tokens  # Return the list of tokens (without joining)

# Apply the tokenize_with_re function to the 'Cleaned Text' column
df['Tokens'] = df['Cleaned Text'].apply(tokenize_with_re)

# Remove 'user' and 'link' from tokens and join words into a single string without extra spaces
df['Tokens'] = df['Tokens'].apply(lambda x: ' '.join([token for token in x if token not in ['user', 'link']]).strip())

# Display the DataFrame with the 'Tokens' column
print(df[['Cleaned Text', 'Tokens']])

                                           Cleaned Text  \
0                     35-årig mand erkender drab [link]   
1     billedserie: hjerteforeningens lokalkomite man...   
2       det må jo være fraregnet #lordbendtner 😉 [link]   
3     det er så godt, @user! psykiatriske disgnoser ...   
4     nøddesbos hånd i herning-rundkørsel: jeg er en...   
...                                                 ...   
1042  @user jeg spørger bare, hvorfor dét er i orden...   
1043                                    så enig! [link]   
1044  @user men det var jo ret sjovt :) som ikke er ...   
1045  dat fejrer 30 år med fly det er i dag 30 år si...   
1046  eriksen har en glimrende statistik mod city [l...   

                                                 Tokens  
0                            35 årig mand erkender drab  
1     billedserie hjerteforeningens lokalkomite mang...  
2                 det må jo være fraregnet lordbendtner  
3     det er så godt , ! psykiatriske disgnoser skal...  
4

In [7]:
#chose not to lemmatise the text since the reference anglifications do not nessescarialy make sense if they are lemmatised, however we can try this if all else fails. 
#the above stopword removal might also make it moredifficult to find regualr expressions for which there might be many in th e anglified dictionary also look at thius when you get further on in the process.

In [8]:
print(df[['text', 'Cleaned Text', 'Tokens']].head())

                                                text  \
0                  35-årig mand erkender drab [LINK]   
1  BILLEDSERIE: Hjerteforeningens lokalkomite man...   
2    Det må jo være fraregnet #LordBendtner 😉 [LINK]   
3  Det er så godt, @USER! Psykiatriske disgnoser ...   
4  Nøddesbos hånd i Herning-rundkørsel: Jeg er en...   

                                        Cleaned Text  \
0                  35-årig mand erkender drab [link]   
1  billedserie: hjerteforeningens lokalkomite man...   
2    det må jo være fraregnet #lordbendtner 😉 [link]   
3  det er så godt, @user! psykiatriske disgnoser ...   
4  nøddesbos hånd i herning-rundkørsel: jeg er en...   

                                              Tokens  
0                         35 årig mand erkender drab  
1  billedserie hjerteforeningens lokalkomite mang...  
2              det må jo være fraregnet lordbendtner  
3  det er så godt , ! psykiatriske disgnoser skal...  
4  nøddesbos hånd i herning rundkørsel jeg er en ..

In [9]:
tweets = df

In [12]:
tweets.to_csv('tweets.csv', index=False)